# Import Dependencies

In [1]:
import os
import numpy as np
import tensorflow as tf
import matplotlib
from matplotlib import pyplot as plt
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers, models, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from sklearn.utils.class_weight import compute_class_weight
from PIL import Image

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus: 
    tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
tf.config.list_physical_devices('GPU')

# Load Data

In [ ]:
data_train_path = '/kaggle/input/balance-everin/Train'
data_val_path = '/kaggle/input/balance-everin/Validation'

In [ ]:
img_width = 256
img_height = 256
batch_size = 32

In [ ]:
data_train = tf.keras.utils.image_dataset_from_directory(
    data_train_path,
    shuffle=True,
    image_size=(img_width, img_height),
    batch_size=batch_size,
    validation_split=0.2,
    subset='training',
    seed=123)

In [ ]:
data_val = tf.keras.utils.image_dataset_from_directory(
    data_train_path,
    shuffle=True,
    image_size=(img_width, img_height),
    batch_size=batch_size,
    validation_split=0.2,
    subset='validation',
    seed=123)

In [ ]:
data_cat = data_train.class_names
print(f"Classes: {data_cat}")

In [ ]:
plt.figure(figsize=(10,10))
for image, labels in data_train.take(1):
    for i in range(9):
        plt.subplot(3,3,i+1)
        plt.imshow(image[i].numpy().astype('uint8'))
        plt.title(data_cat[labels[i]])
        plt.axis('off')

# Modelling from Scratch 

In [ ]:
data_train

In [ ]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal_and_vertical"),
    tf.keras.layers.RandomRotation(0.3),
    tf.keras.layers.RandomZoom(0.3),
    tf.keras.layers.RandomContrast(0.2),
    tf.keras.layers.RandomBrightness(0.2)
])

In [ ]:
model = tf.keras.Sequential([
    data_augmentation,
    tf.keras.layers.Rescaling(1./255),
    tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(len(data_cat), activation='softmax')
])

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

In [ ]:
epochs_size = 25
history = model.fit(
    data_train,
    validation_data=data_val,
    epochs=epochs_size,
)

In [ ]:
# Plot Accuracy
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Plot Loss
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

## Make Prediction

In [ ]:
image = '/kaggle/input/baksoaa/bakso33.jpeg'
image = tf.keras.utils.load_img(image, target_size=(img_height,img_width))
img_arr = tf.keras.utils.array_to_img(image)
img_bat=tf.expand_dims(img_arr,0)

In [ ]:
predict = model.predict(img_bat)

In [ ]:
score = tf.nn.softmax(predict)

In [ ]:
print('The image is {} with accuracy of {:0.2f}'.format(data_cat[np.argmax(score)],np.max(score)*100))

Prediction still incorrect

# Transfer Learning

In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense, Rescaling
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [ ]:
base_model = tf.keras.applications.MobileNetV2(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3)
)

In [ ]:
base_model.trainable = False

In [ ]:
model = tf.keras.Sequential([
    # Pretrained Feature Extractor
    base_model,
    
    # Global Average Pooling
    tf.keras.layers.GlobalAveragePooling2D(),

    # Fully Connected Layers
    tf.keras.layers.Dropout(0.5),  # Dropout untuk regularisasi
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),  # Dropout tambahan
    tf.keras.layers.Dense(len(data_cat), activation='softmax')  # Output layer
])

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

In [ ]:
model.summary()

In [ ]:
lr_schedule = tf.keras.callbacks.LearningRateScheduler(
    lambda epoch: 0.001 * 0.95 ** epoch, verbose=1
)

In [ ]:
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=10, restore_best_weights=True
)

In [ ]:
train_data = tf.keras.preprocessing.image_dataset_from_directory(
    "/kaggle/input/balance-everin/Train",
    image_size=(224, 224),
    batch_size=32
)

In [ ]:
val_data = tf.keras.preprocessing.image_dataset_from_directory(
    "/kaggle/input/balance-everin/Validation",
    image_size=(224, 224),
    batch_size=32
)

In [ ]:
normalization_layer = tf.keras.layers.Rescaling(1./255)
train_data = train_data.map(lambda x, y: (normalization_layer(x), y))
val_data = val_data.map(lambda x, y: (normalization_layer(x), y))

In [ ]:
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=50,
    callbacks=[lr_schedule, early_stop]
)

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.title('Accuracy Curve')
plt.show()

plt.figure(figsize=(10, 5))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.title('Loss Curve')
plt.show()

In [ ]:
val_loss, val_accuracy = model.evaluate(val_data)
print(f"Validation Accuracy: {val_accuracy:.2f}, Validation Loss: {val_loss:.2f}")

In [ ]:
# Simpan model transfer learning dalam format .h5
model.save("mobilenetv2_transfer_learning.h5")

# Fine Tuning

In [ ]:
base_model.trainable = True

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

In [ ]:
fine_tune_lr_schedule = tf.keras.callbacks.LearningRateScheduler(
    lambda epoch: 1e-5 * 0.95 ** epoch, verbose=1
)

In [ ]:
history_fine = model.fit(
    train_data,
    validation_data=val_data,
    epochs=20,  # Epoch lebih sedikit untuk fine-tuning
    callbacks=[fine_tune_lr_schedule, early_stop]
)

In [ ]:
val_loss, val_accuracy = model.evaluate(val_data)
print(f"Fine-tuned Validation Accuracy: {val_accuracy}, Validation Loss: {val_loss}")

In [ ]:
model.save("mobilenetv2_fine_tuned.h5")

# Predict

In [ ]:
import tensorflow as tf
from tensorflow.keras.utils import load_img, img_to_array

# Load model
model = tf.keras.models.load_model("/kaggle/working/mobilenetv2_fine_tuned.h5")

# Load gambar
image_path = "/kaggle/input/baksoaa/bakso33.jpeg"
img = load_img(image_path, target_size=(224, 224))
img_array = img_to_array(img)
img_array = img_array / 255.0
img_array = img_array[None, ...]

predictions = model.predict(img_array)
predicted_class = tf.argmax(predictions, axis=1).numpy()[0]

class_indices = {
    0: 'Abc Kopi Susu', 1: 'Acaraki', 2: 'Apel', 3: 'Bakso', 4: 'Beef Teriyaki', 5: 'Beng Beng',
    6: 'Chocopie', 7: 'Coca Cola', 8: 'Durian', 9: 'Egg Roll', 10: 'Fanta', 11: 'Fitbar',
    12: 'Floridina', 13: 'Golda Dolce', 14: 'Ice Cream', 15: 'Indomie Goreng', 16: 'Iso Plus',
    17: 'Japota', 18: 'Jeruk', 19: 'Kitkat', 20: 'Mie Ayam', 21: 'Milo', 22: 'Nasi Padang',
    23: 'Nasi Uduk', 24: 'Nasi goreng', 25: 'Nescafe Ice Black', 26: 'Nipis Madu',
    27: 'Nu Green Tea', 28: 'Oat Milk', 29: 'Olatte', 30: 'Onigiri', 31: 'Oreo', 32: 'Pacar Cina',
    33: 'Pisang', 34: 'Pocari Sweat', 35: 'Pocky Chocolate', 36: 'Rendang', 37: 'Root Beer Kaleng',
    38: 'Roti Tawar', 39: 'Sarigandum', 40: 'Semangka', 41: 'Sprite', 42: 'Sushi',
    43: 'Tahu Goreng', 44: 'Tango', 45: 'Taro', 46: 'Teh Botol', 47: 'Teh Kotak',
    48: 'Ultra Milk', 49: 'You C Orange', 50: 'Yougurt'
}

predicted_label = class_indices.get(predicted_class, "Unknown")
print(f"Predicted Class Index: {predicted_class}")
print(f"Predicted Label: {predicted_label}")

# Convert TFlite

In [ ]:
model = tf.keras.models.load_model("/kaggle/input/v1/tensorflow2/default/1/v1.h5")

converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open("v1.tflite", "wb") as f:
    f.write(tflite_model)